In [1]:
import os

ROOT_DATA = os.environ.get('LARD_DATA_ROOT_PATH')
ROOT_PROJECT = os.environ.get('LARD_PROJECT_ROOT_PATH')

print(ROOT_DATA)
print(ROOT_PROJECT)


/fileonssd/runway-dataset/lard-dataset
/home/yeli/workspace/lard/lard-detection


# 复杂度计算

In [83]:
from ultralytics import YOLO
import pandas as pd
import os
import glob
from IPython.display import clear_output


# yaml中的nc要符合实际
def get_complex(path_yaml, imgsz=640):
    model = YOLO(path_yaml, task='detect')
    n_l, n_p, n_g, flops = model.model.info(imgsz=imgsz)
    return n_l, n_p, n_g, flops


In [10]:
# 参数定义
all_path_yaml = glob.glob(f'{ROOT_PROJECT}/cfg/ultralytics/models/yolov8n-p2/*.yaml')


# 计算复杂度
all_complex = []
for path_yaml in all_path_yaml:
    filename = os.path.basename(path_yaml).split('.yaml')[0]
    _, _, _, flops_640 = get_complex(path_yaml, imgsz=640)
    n_l, n_p, n_g, flops_1280 = get_complex(path_yaml, imgsz=1280)
    all_complex.append([filename, n_p, flops_640, flops_1280])
    clear_output(wait=True)
    

# 结果重构
df = pd.DataFrame(all_complex, columns=['model', 'param', 'gflops_640', 'gflops_1280'])
df = df.sort_values(['gflops_640', 'gflops_1280'], ascending=False).reset_index(drop=True)
df.param = df.param * 1e-6
df.to_csv(f'{ROOT_PROJECT}/results/tables/info/complex.csv', index=False)
display(df.round(3))

,model,param,gflops_640,gflops_1280
0,yolov8n-p2_cbam_afpn,6.513,63.527,254.106
1,yolov8n-p2_afpn,6.447,63.474,253.895
2,yolov8n-p2_cbam_afpn_small,3.158,24.977,99.907
3,yolov8n-p2_afpn_small,3.092,24.924,99.695
4,yolov8n-p2_gam,4.994,18.746,74.985
5,yolov8n-p2_vit,5.192,18.273,73.094
6,yolov8n-p2_lsk_bifpn_carafe,3.573,17.940,71.762
7,yolov8n-p2_carafe,3.434,17.742,70.966
8,yolov8n-p2_cpca,3.481,17.665,70.662
9,yolov8n-p2_lsk_bifpn,3.493,17.633,70.534


In [5]:
# todo mmdet mmyolo 复杂度计算